In [1]:
import os

In [2]:
os.listdir()

['matrixTweetsEmb_4177_100_50_300.dump',
 '.DS_Store',
 'botNotBot.ipynb',
 '02.CNN_100x50x300D_google_0.9693.h5',
 'botNotBot.py',
 'Dataset-README',
 '.ipynb_checkpoints',
 '01.CNN_100x50x300D_google_0.964.h5',
 'en',
 '.git',
 'matrixTweetsEmb_Compressed.dump']

In [3]:
pathEn = "/Users/kram/Downloads/botOrNot-en_es/en"

First of all, we will create a procedure to be tested on a single file.

After this first step will be completed, we will extend this procedure to create a complete dataframe

In [4]:
testfile = "/Users/kram/Downloads/botOrNot-en_es/en/1a5b808546838869bc39cebdbad951e3.xml"

In [5]:
import pandas as pd
import xml.etree.ElementTree as ET
import io

def iter_docs(author):
    '''This function extracts the text and the language from the XML'''
    author_attr = author.attrib
    for doc in author.iter('document'):
        doc_dict = author_attr.copy()
        doc_dict.update(doc.attrib)
        doc_dict['data'] = doc.text
        yield doc_dict

xml_data = open(testfile, "r") # Opening the text file
etree = ET.parse(xml_data) # Create an ElementTree object 
df = pd.DataFrame(list(iter_docs(etree.getroot()))) #Append the info to a pandas dataframe

In [6]:
df.head(10)

,data,lang
0,1:7 Wherefore she went after their families: o...,en
1,And he put his hand over the host: and they ga...,en
2,65:18 But be ye far from the Philistines.,en
3,"4:29 And rose up, and went out.",en
4,"24:13 My son, keep my mouth hath spoken, sayin...",en
5,30:21 And afterwards she bare unto him.,en
6,"And on the ground, and took Rebekah, and said ...",en
7,13:10 And I the LORD came unto Elim: and in th...,en
8,And his servants for his issue.,en
9,20:3 And cast him down: deliver my people shal...,en


In [7]:
df['data'][0]

'1:7 Wherefore she went after their families: of Sered, the family of the priests, and the light shine upon thy head; for I fear the LORD.'

In [8]:
# Getting ID to insert in the dataframe

filename = testfile.split("/")[-1].split(".")[0]

In [9]:
filename

'1a5b808546838869bc39cebdbad951e3'

We can now try to extend the procedure to the full directory.

In [10]:
import time

# Creating empty dataframe
dataEn = pd.DataFrame()

# Monitoring time to load the files
start = time.time()

for root, dirs, files in os.walk(pathEn):
    for file in files:
        if file == 'truth.txt':
            continue
        else: 
            try:
                pathToFile = root + '/' + file # Creating path
                # print(pathToFile) # Just for debugging
                xml_data = open(pathToFile, "r", encoding="utf8") # Opening the text file
                etree = ET.parse(xml_data) # Create an ElementTree object
                data = list(iter_docs(etree.getroot())) # Create a list of dictionaries with the data
                filename = file.split(".")[0] # Get filename
                for dictionary in data: # Loop through the dictionary
                    dictionary['ID'] = filename # Append filename
                dataEn = dataEn.append(data)  # Append the list of dictionary to a pandas dataframe
                
            # If the file is not valid, skip it
            except ValueError as e:
                print(e)
                continue
            
end = time.time()
print("Total running time is", end - start)

Total running time is 109.33368396759033


In [11]:
dataEn.head(10)

,ID,data,lang
0,867be96f95dfc4e24541d19c6a5ab8bf,Alex is too nice for love island :(,en
1,867be96f95dfc4e24541d19c6a5ab8bf,RT @STVNews: Teenager charged with rape of wom...,en
2,867be96f95dfc4e24541d19c6a5ab8bf,@LipsTaco @jennyhastie,en
3,867be96f95dfc4e24541d19c6a5ab8bf,@RyanDunbar8 happy bday Ryan have the best day...,en
4,867be96f95dfc4e24541d19c6a5ab8bf,@jennyhastie @bootywhispers I just wanna let j...,en
5,867be96f95dfc4e24541d19c6a5ab8bf,@bootywhispers @jennyhastie what would u do ?,en
6,867be96f95dfc4e24541d19c6a5ab8bf,@Aidsterrr Asking for a mate :/ wee bit line o...,en
7,867be96f95dfc4e24541d19c6a5ab8bf,Wonder what a Sunday without the fear feels like,en
8,867be96f95dfc4e24541d19c6a5ab8bf,RT @liamfrenchx: Brutal getn wee flashbacks a ...,en
9,867be96f95dfc4e24541d19c6a5ab8bf,Went to a&amp;e last night myself after I lock...,en


In [12]:
dataEn['data'][0]

0                  Alex is too nice for love island :(
0    RT @AlexanderRugaev: The Crypto Finance Ecosys...
0    Check out these awesome cooking t-shirts &amp;...
0    YEWWinfo Tiny Nanoparticles to Treat a Huge Pr...
0    Sr. Project Manager Water / Wastewater Enginee...
0                      @lolzdonz @beckyfrancesxo !!!!!
0    @jennycastle96 Ahaha last time acting reckless 😂😂
0    Apr. 20/2002 - The brodway show Jesus Christ S...
0    I'm so excited for the boojum I'm about to get...
0    Killer bonus here 70+ page free download to he...
0                        boooo https://t.co/Q32Ttd9FIH
0    Create a life you love.\n\n#quote #life https:...
0    @puretemerity The archaeology on Arran is amaz...
0    Why Silicon Valley can’t fix itself\n\nhttps:/...
0    @PayChen @designtaxi So you have place to put ...
0    #GameofThrones #ThorosofMyr: There's no story....
0    the amount of pride flags he has around him, a...
0    @mike_mcgrail Pretty sure I seen your doppelga...
0        M

In [13]:
dataEn.describe()

,ID,data,lang
count,412000,412000,412000
unique,4120,373908,1
top,db91aa8f939b1c18a38510f589e7231d,"wow, it's so sad about MJ :(",en
freq,100,136,412000


Now that we have merged the IDs with the data, we can create another dataframe with the labels and then merge them using the ID as key

In [14]:
pathToLabels = "/Users/kram/Downloads/botOrNot-en_es/en/truth.txt"

In [15]:
target = pd.read_csv(pathToLabels, sep=":::")
target.columns=['ID', 'botOrHuman', 'sex'] 

/Users/kram/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [16]:
target.head(10)

,ID,botOrHuman,sex
0,2bed15d46872169dc7deaf8d2b43a56,bot,bot
1,25395a7dbd2caa3d828bb3dbd57d8857,bot,bot
2,1c9f161414334b286c4dc70163744390,bot,bot
3,1da1f87b3dc778f28268eec70ce94f19,bot,bot
4,bd4286bfafb8a35b8e132a396b884e07,bot,bot
5,af61c4c017f246da69285497baf3dc0b,bot,bot
6,7a36090b20e7bddbe55561c52f959041,bot,bot
7,5183ff5bedcab9a6a301ff04e27166cd,bot,bot
8,6ac10734f35a773e9f2209f8668fffdf,bot,bot
9,b5b06752b42d3c354cc8e126f030a864,bot,bot


In [17]:
target.describe()

,ID,botOrHuman,sex
count,4119,4119,4119
unique,4119,2,3
top,cf387a26c11e05174cb50fbea00223b0,human,bot
freq,1,2060,2059


We can now proceed with the concatenation of the dataframes for the English language

In [18]:
mergedEnData = pd.merge(dataEn, target, on='ID')

In [19]:
mergedEnData.head(10)

,ID,data,lang,botOrHuman,sex
0,867be96f95dfc4e24541d19c6a5ab8bf,Alex is too nice for love island :(,en,human,female
1,867be96f95dfc4e24541d19c6a5ab8bf,RT @STVNews: Teenager charged with rape of wom...,en,human,female
2,867be96f95dfc4e24541d19c6a5ab8bf,@LipsTaco @jennyhastie,en,human,female
3,867be96f95dfc4e24541d19c6a5ab8bf,@RyanDunbar8 happy bday Ryan have the best day...,en,human,female
4,867be96f95dfc4e24541d19c6a5ab8bf,@jennyhastie @bootywhispers I just wanna let j...,en,human,female
5,867be96f95dfc4e24541d19c6a5ab8bf,@bootywhispers @jennyhastie what would u do ?,en,human,female
6,867be96f95dfc4e24541d19c6a5ab8bf,@Aidsterrr Asking for a mate :/ wee bit line o...,en,human,female
7,867be96f95dfc4e24541d19c6a5ab8bf,Wonder what a Sunday without the fear feels like,en,human,female
8,867be96f95dfc4e24541d19c6a5ab8bf,RT @liamfrenchx: Brutal getn wee flashbacks a ...,en,human,female
9,867be96f95dfc4e24541d19c6a5ab8bf,Went to a&amp;e last night myself after I lock...,en,human,female


In [20]:
mergedEnData.describe()

,ID,data,lang,botOrHuman,sex
count,411900,411900,411900,411900,411900
unique,4119,373808,1,2,3
top,db91aa8f939b1c18a38510f589e7231d,"wow, it's so sad about MJ :(",en,human,bot
freq,100,136,411900,206000,205900


# Deep Learning - CNN?

In [21]:
'''Creo la litsa degli ID, delle classi e dei tweets pr ogni ID'''

listaIds =[]
listaClasses = []
matrixTweets = []

for index, x in mergedEnData.iterrows():
    id = x['ID']
    if id not in listaIds:
        newList = list()
        newList.append(x[1])
        matrixTweets.append(newList)
        listaIds.append(id)
        listaClasses.append(x[3])
    else:
        ls = matrixTweets[listaIds.index(id)]
        ls.append(x[1])
        matrixTweets[listaIds.index(id)] = ls
        
print(len(listaIds))

4119


In [22]:
'''Trasformo le entità, lascio le faccine, levo le stopword e se serve agli embeddings lemmatizzo'''

'Trasformo le entità, lascio le faccine, levo le stopword e se serve agli embeddings lemmatizzo'

In [23]:
import sys
!{sys.executable} -m pip install ekphrasis

You are using pip version 18.1, however version 19.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [24]:
from ekphrasis.dicts.emoticons import emoticons
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer

text_processor = TextPreProcessor (
    # terms that will be normalized
    normalize=[ 'email' , 'percent' , 'money' , 'phone' ,
                'time' , 'url' , 'date' , 'number' ] ,
    fix_html=True ,  # fix HTML tokens
    segmenter="twitter" ,
    corrector="twitter" ,
    unpack_hashtags=True ,  # perform word segmentation on hashtags
    unpack_contractions=True ,  # Unpack contractions (can't -> can not)
    spell_correct_elong=True ,  # spell correction for elongated words
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    dicts=[ emoticons ]
)

Reading twitter - 1grams ...
Reading twitter - 2grams ...
Reading twitter - 1grams ...


In [25]:
'''Transform sentences to word embeddings'''

'Transform sentences to word embeddings'

In [26]:
import gensim

In [27]:
google_300 = gensim.models.KeyedVectors.load_word2vec_format( "/Users/kram/Downloads/botOrNot-en_es/google_w2v_300.bin" , binary=True )


In [28]:
'''Trasformo le frasi'''

'Trasformo le frasi'

In [32]:
import numpy as np
from nltk.tokenize import TweetTokenizer as TweetTokenizer
from nltk.corpus import stopwords
import random as rn
stop_words = set(stopwords.words('english'))
i = 0
matrixTweetsEmb = []
for tweetsUser in matrixTweets:
    embTweetsUser = []
    if(i % 100) == 0:
         print(i)
    for tweet in tweetsUser:
        embTweetUser = np.zeros([50,300], dtype=np.float16)
        #Preprocesso
        tokList = text_processor.pre_process_doc(tweet)
        #Rimuovo le stopwords
        tokList = [w for w in tokList if not w in stop_words]
        #trovo l'embedding
        numTok = 0;
        for token in tokList[0:50]:
            g_vec =[]
            is_in_model = False
            if token in google_300.vocab.keys ( ):
                is_in_model = True
                g_vec = google_300.word_vec(token)
            elif token == "<number>":
                is_in_model = True
                g_vec = google_300.word_vec( "number")
            elif token == "<percent>":
                is_in_model = True
                g_vec = google_300.word_vec("percent")
            elif token == "<money>":
                is_in_model = True
                g_vec = google_300.word_vec("money")
            elif token == "<email>":
                is_in_model = True
                g_vec = google_300.word_vec("email")
            elif token == "<phone>":
                is_in_model = True
                g_vec = google_300.word_vec("phone")
            elif token == "<time>":
                is_in_model = True
                g_vec = google_300.word_vec("time")
            elif token == "<date>":
                is_in_model = True
                g_vec = google_300.word_vec("date")
            elif token == "<url>":
                is_in_model = True
                g_vec = google_300.word_vec("url")
            elif not is_in_model:
                max = len ( google_300.vocab.keys ( ) ) - 1
                index = rn.randint ( 0 , max )
                word = google_300.index2word[ index ]
                g_vec = google_300.word_vec( word )

            embTweetUser[numTok] = np.array(g_vec, dtype=np.float16)
            numTok += 1
        embTweetsUser.append(np.array(embTweetUser, dtype=np.float16))
    i +=1
    matrixTweetsEmb.append(np.array(embTweetsUser, dtype=np.float16))


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100


In [33]:
'''Num Utenti x Num Tweets x Num MaxTokens x Dim Embedding'''
import numpy as np 
matrixTweetsEmb = np.array(matrixTweetsEmb, dtype=np.float16)
print(matrixTweetsEmb.shape)

(4119, 100, 50, 300)


In [34]:
#!{sys.executable} -m pip install joblib
import joblib
joblib.dump(matrixTweetsEmb,'matrixTweetsEmb_ALT.dump')
#matrixTweetsEmb = joblib.load('matrixTweetsEmb_4177_100_50_300.dump')
joblib.dump(listaClasses,'listaClasses.dump')

['listaClasses.dump']

In [5]:
from keras.layers import *
from keras.models import Sequential
model = Sequential()
model.add(Conv2D(200,(5,5), activation ='relu', input_shape=(100,50,300)))
model.add(MaxPool2D(2,2))
model.add(Conv2D(100,(5,4), activation ='relu'))
model.add(MaxPool2D(2,2))
model.add(Conv2D(20,(3,3), activation ='relu'))
model.add(MaxPool2D(2,2))
model.add(Flatten())
model.add(Dense(400, activation="tanh"))
model.add(Dense(200, activation="tanh"))
model.add(Dense(100, activation="tanh"))
model.add(Dense(2, activation="softmax"))
model.summary()

/Users/kram/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 96, 46, 200)       1500200   
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 48, 23, 200)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 44, 20, 100)       400100    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 22, 10, 100)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 20, 8, 20)         18020     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 10, 4, 20)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 800)               0         
__________

In [32]:
#!{sys.executable} -m pip install category_encoders
import category_encoders as ce
le =  ce.OneHotEncoder(return_df=False, impute_missing=False, handle_unknown="ignore")
training_classes = le.fit_transform(listaClasses)
print(le.category_mapping)


[{'col': 0, 'mapping': [('human', 1), ('bot', 2)]}]


In [27]:
model.compile ( loss='categorical_crossentropy' , optimizer='adam' , metrics=['accuracy'] )

history = model.fit(matrixTweetsEmb,training_classes,64,15,
                      validation_split= 0.15 ,
                      verbose=1)

Train on 3501 samples, validate on 618 samples
Epoch 1/15
3501/3501 [==============================] - 1768s 505ms/step - loss: 0.4883 - acc: 0.7721 - val_loss: 0.2417 - val_acc: 0.9061
Epoch 2/15
3501/3501 [==============================] - 1666s 476ms/step - loss: 0.2040 - acc: 0.9292 - val_loss: 0.2187 - val_acc: 0.9369
Epoch 3/15
3501/3501 [==============================] - 1653s 472ms/step - loss: 0.1142 - acc: 0.9614 - val_loss: 0.1788 - val_acc: 0.9369
Epoch 4/15
3501/3501 [==============================] - 1658s 474ms/step - loss: 0.0870 - acc: 0.9694 - val_loss: 0.1753 - val_acc: 0.9417
Epoch 5/15
3501/3501 [==============================] - 1659s 474ms/step - loss: 0.0437 - acc: 0.9843 - val_loss: 0.1882 - val_acc: 0.9531
Epoch 6/15
3501/3501 [==============================] - 1657s 473ms/step - loss: 0.0201 - acc: 0.9937 - val_loss: 0.1714 - val_acc: 0.9547
Epoch 7/15
3501/3501 [==============================] - 1657s 473ms/step - loss: 0.0209 - acc: 0.9934 - val_loss: 0.137

In [31]:
import matplotlib.pyplot as plt
print(history.history.keys())
%matplotlib qt
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

dict_keys(['val_loss', 'val_acc', 'loss', 'acc'])


In [40]:
model.save('02.CNN_100x50x300D_google_0.9693.h5')

In [33]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(matrixTweetsEmb,training_classes, test_size=0.15, random_state=891)

In [36]:
from  sklearn.metrics  import classification_report
from keras.callbacks import Callback
class MyCallBack(Callback):
    def __init__(self,verbose=0):

        super(Callback, self).__init__()
        self.verbose = verbose

    def on_epoch_end(self, epoch, logs={}):
        current = logs.get('val_loss')
        # if current < 0.014:
        #     self.model.stop_training = True

        predicted = model.predict ( X_test )

        test = [ '0' ] * len ( X_test )
        i = 0
        for cl in predicted:
            test[ i ] = str ( np.argmax ( cl ) )
            i += 1

        test_lab = [ '0' ] * len ( X_test )
        i = 0
        for cl in y_test:
            test_lab[ i ] = str ( np.argmax ( cl ) )
            i += 1

        print ( len ( X_test ) )
        print ( classification_report ( test , test_lab ) )
        
        
callbacks_list = [
    MyCallBack(verbose=1)
]


In [38]:
model.compile ( loss='categorical_crossentropy' , optimizer='adam' , metrics=['accuracy'] )

history = model.fit(X_train,y_train,64,15,
                      validation_data= (X_test,y_test) ,
                      callbacks=callbacks_list,
                      verbose=1)

Train on 3501 samples, validate on 618 samples
Epoch 1/15
3501/3501 [==============================] - 1826s 522ms/step - loss: 0.3299 - acc: 0.8855 - val_loss: 0.1894 - val_acc: 0.9369
618
             precision    recall  f1-score   support

          0       0.96      0.91      0.93       299
          1       0.92      0.97      0.94       319

avg / total       0.94      0.94      0.94       618

Epoch 2/15
3501/3501 [==============================] - 1657s 473ms/step - loss: 0.1466 - acc: 0.9532 - val_loss: 0.2685 - val_acc: 0.9126
618
             precision    recall  f1-score   support

          0       0.83      0.97      0.90       240
          1       0.98      0.87      0.92       378

avg / total       0.92      0.91      0.91       618

Epoch 3/15
3501/3501 [==============================] - 1643s 469ms/step - loss: 0.1137 - acc: 0.9597 - val_loss: 0.1312 - val_acc: 0.9644
618
             precision    recall  f1-score   support

          0       0.98      0.94      0.

KeyboardInterrupt: 

In [41]:
from sklearn.metrics import accuracy_score
predicted = model.predict(X_test)

test = ['0']* len(X_test)
i= 0
for cl in predicted:
    test[i] = str(np.argmax (cl))
    i +=1

test_lab = ['0']* len(X_test)
i= 0
for cl in y_test:
    test_lab[ i ] = str(np.argmax ( cl ))
    i += 1


print(len(X_test))
acc = accuracy_score(test, test_lab)
print(" Accuracy:", acc)

print(classification_report(test,test_lab))

618
 Accuracy: 0.9676375404530745
             precision    recall  f1-score   support

          0       0.97      0.96      0.96       284
          1       0.97      0.97      0.97       334

avg / total       0.97      0.97      0.97       618



In [43]:
import zlib
compressed = zlib.compress(matrixTweetsEmb)


TypeError: dump() missing 1 required positional argument: 'filename'

In [44]:
joblib.dump(compressed,'matrixTweetsEmb_Compressed.dump')

['matrixTweetsEmb_Compressed.dump']